In [ ]:
"""
Caleb Ellington
ML 10701, Fall 2020
Final Project

Default VAE
"""

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models

# AWS
import sagemaker
import s3fs

In [2]:
# load data from s3
role = sagemaker.get_execution_role()
fs = s3fs.S3FileSystem()
bucket = 'cellingt-ml-final-project'
subfolder = 'data/FER/'
path = f's3://{bucket}/{subfolder}'

dataset = np.load(fs.open(f's3://{bucket}/FER_processed.npz'))
xtrain, ytrain, xtest, ytest = dataset['xtrain'], dataset['ytrain'], dataset['xtest'], dataset['ytest']
ylabels = {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'sad': 4, 'surprise': 5, 'neutral': 6}
ylabels_rev = {val:key for key, val in ylabels.items()}

In [20]:
# Check testset duplicates
duplabels = []
print(len(xtest))
for i, (img1, y1) in enumerate(zip(xtest[:-1], ytest[:-1])):
    if i%100 == 0:
        print(i, len(duplabels))
    for j, (img2, y2) in enumerate(zip(xtest[i+1:], ytest[i+1:])):
        if (img1 == img2).all():
            duplabels.append((i, i+j+1, y1, y2))
dups = [(i, j) for i, j, _, _ in duplabels]
np.save(fs.open(f's3://{bucket}/FER_testset_dups.npy', 'wb'), dups)

7178
0 0
100 0
200 1
300 1
400 1
500 2
600 2
700 4
800 5
900 6
1000 6
1100 6
1200 7
1300 7
1400 7
1500 8
1600 8
1700 8
1800 8
1900 8
2000 9
2100 11
2200 11
2300 11
2400 11
2500 14
2600 14
2700 14
2800 15
2900 16
3000 16
3100 16
3200 17
3300 19
3400 23
3500 24
3600 24
3700 25
3800 27
3900 28
4000 28
4100 32
4200 38
4300 51
4400 57
4500 61
4600 64
4700 64
4800 66
4900 68
5000 68
5100 68
5200 73
5300 73
5400 74
5500 74
5600 74
5700 75
5800 75
5900 75
6000 75
6100 75
6200 77
6300 82
6400 83
6500 83
6600 84
6700 84
6800 84
6900 87
7000 89
7100 90


91